#### REAL TIME PLOTTING FOR PRESSURE & CHECKING RECORDING CONFIG ####

## STEP 1: LOAD DEPENDENCIES, SET GLOBAL PARAMETERS, DEFINE FUNCTIONS

In [1]:
# Import Libraries and Library Settings
import numpy as np
import scipy.io as sio
from os.path import dirname, join as pjoin
import time
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib tk
from matplotlib.animation import FuncAnimation
import glob
import os
import re
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

# Setting Params for plotting & troubleshooting
np.set_printoptions(threshold=np.inf)
plt.rcParams['animation.html']='jshtml'

# Define Functions in Use
def func(p):
    return int(((p.split('.')[0]).split('_'))[-1])

# For a histogram plotted via grid, add labels to the top of each bar
def addlabels(x,y,ax):
    x2 = x[:-1]
    for i in range(len(x2)):
        ax.annotate(y[i],(x2[i]+(x[1]-x[0])/2,y[i]+5),ha='center',color='r',fontsize=9)
        
# If you have a channel's row and col, return the channel's index
def map2idx(ch_row,ch_col):
    if ch_row > 31 or ch_row <0:
        print('Row out of range')
    elif ch_col >31 or ch_col<0:
        print('Col out of range')
    else:
        ch_idx = int(ch_row*32 + ch_col)
    return ch_idx

# If you have a channel index, return the channel's row and col
def idx2map(ch_idx):
    if ch_idx > 1023 or ch_idx < 0:
        print('Chan num out of range')
    else:
        ch_row = int(ch_idx/32)
        ch_col = int(ch_idx - ch_row*32)
    return (ch_row,ch_col)

## STEP 2: INITIALIZE CHANGING VARIABLES

In [2]:
piece_no = 0
time_win = 20 #The amount of time (ms) you want to view in each frame 
real_time_win = 80 #The amount of time (ms) you want to view in the current buffer
max_samples = 200000 #To minimize plot delays, set max samples to plot for single electrodes
live = False # True / False
mat_var = 'gmem2' #String of the variable name in the mat file of the FPGA 'vect' 'gmem1''gmem2'
non_live_datarun = 10 # If looking at old data, enter in the datarun number
non_live_date_piece = '2021-08-23-0' # If looking at old data, enter in the date & piece
save = False # The resulting noise matrix and noise values, True / False

## STEP 3: DETERMINE PATHS, INITIALIZE  PLOTS, INITIALIZE VARIABLES, & FIRST CHECKS

In [3]:
# Initialize the Path and Data Name
path = r'/Shakedown/Users/vision/Desktop/Xilinx'

# Determine Folder and Datarun if Live Data
if live == True:
    
    date_piece = datetime.today().strftime('%Y-%m-%d') + '-' + str(piece_no)

    # Update the dataXXX folder to the latest one (assumes you want to plot noise tests happening now)
    if os.path.isdir(date_piece):
        dataruns = os.listdir(date_piece)
        last_datarun = 0
        for dir in dataruns:
            if dir.startswith('data'):
                val = int(re.findall('\d+',dir)[0])

                if val > last_datarun:
                    last_datarun = val

        run = last_datarun + 1

    else:
        run = 0

# Determine Folder and Datarun if Previously Recorded Data
elif live != True:
    date_piece = non_live_date_piece
    run = non_live_datarun

datarun = 'data'+str(run).zfill(3)
file_type = '/*mat'


buffer_num = 0
latest_file_pre = pjoin(date_piece,datarun,datarun+'_'+str(buffer_num-1)+'.mat')

squish = 3

# Initialize Figure and Properties
# Figure
plt.close('all')
fig2, ax2 = plt.subplots(1,1,facecolor='white',figsize=(2.5,2.5))
textstr3 = 'GUI CONTROLS\n\n'
textstr4 = 'Pan/Zoom: Toggle [p], Click [l/r], Drag\nSave: [s]\nConstrain Pan/Zoom: Hold [x/y]\nPreserve Aspect Ratio: Hold [ctrl]\nReset: [r]'
ax2.text(0.1,0.5,textstr3+textstr4,transform=ax2.transAxes,fontsize=9,va = 'center', ha = 'left', bbox={'facecolor':'white','edgecolor':'black','linewidth':1.5})
ax2.axis('off')
fig2.show()

fig, ax = plt.subplots(2,3,figsize=(20,12),facecolor='white',gridspec_kw={'height_ratios':[2,1.5],'width_ratios':[2,1.5,4]})
fig.show()
plt.style.use('fivethirtyeight')

# Subplots
plt.subplots_adjust(left=0.05,right=0.975,bottom=0.05,top=0.99,wspace=0.2,hspace=0.01)

# Aesthetics
ax[0,1].set_title('Current Elec Max Samples',{'fontsize':14})
ax[1,1].set_title('Current Elec Max Amplitude',{'fontsize':14})
ax[1,0].set_title('Unique Channels per Sample',{'fontsize':14})
twin1 = ax[1,0].twinx()
ax[0,2].set_title('Sample Rate',{'fontsize':14})
ax[1,2].set_title('Amplitude',{'fontsize':14})

textstr2 = 'Time Window = %.2f ms' %(time_win)
ax[0,2].text(1, 1.15, textstr2,transform=ax[0,2].transAxes,fontsize=12,va = 'center',ha='right')
ax[0,0].set_xlim(left=-2,right=34)
ax[0,0].set_ylim(bottom=-2,top=34)
ax[0,1].set_xlabel('Time (ms)',{'fontsize':12})
ax[0,1].set_ylabel('Amplitude (ADC)',{'fontsize':12})
ax[1,1].set_xlabel('Time (ms)',{'fontsize':12})
ax[1,1].set_ylabel('Amplitude (ADC)',{'fontsize':12})
ax[1,0].set_xlabel('Unique Channel Count',{'fontsize':12})
ax[1,0].set_ylabel('Number of Samples',{'fontsize':12})
twin1.set_ylabel('Number of Buffers',{'fontsize':12,'rotation':-90,'va':'bottom'},loc='top')
ax[0,2].set_xlabel('Electrode Number',{'fontsize':12})
ax[0,2].set_ylabel('Avg. Sample Count per Buffer',{'fontsize':12})
ax[1,2].set_xlabel('Electrode Number',{'fontsize':12})
ax[1,2].set_ylabel('Avg. Amplitude per Sample',{'fontsize':12})


# Initialize Figure Variables
max_dot = 600
sample_win = int(time_win/0.05)
last_time = 0 # Overall
last_time1 = 0 #Samp
last_time2 = 0 #Amp
cnt_time1 = 0 #Samp
cnt_time2 = 0 #Amp

# Initialize Data Conversion & Analysis Variables
chan_num_pre = -1
buffer_num = 0

sam_per = np.zeros((32,32))
amp_per = np.zeros((32,32))
unq_per = np.zeros((0,0))
avg_unq = np.zeros((1,1))
amp_cnt = np.zeros((32,32))
buf_cnt = 0
fig_elec = np.zeros((2,3))
fig_elec_pre = np.zeros((2,1))

## STEP 4: PRE-LOOP HOLDS 

# If the file does not yet exist, stay here
while not(os.path.isdir(pjoin(date_piece, datarun))):
    continue

# If the file exists, but hasn't yet been filled, hold here
while True:
    data_dir = glob.glob(pjoin(date_piece,datarun)+file_type)
    if len(data_dir)>0:
        break
        


## STEP 5: CONTINUOUS DATA ANALYSIS & PLOTTING

In [10]:
# Continuously Check for New Data
while True:
    data_dir = glob.glob(pjoin(date_piece,datarun)+file_type)
    
    # Choose the file by the largest file number
    latest_file = max(data_dir, key=func)
    
    # If the latest file is the same as the previously plotted file, hold
    if live == True:
        if latest_file == latest_file_pre:
            continue
    
    print(latest_file)
    
    # In case the latest file is just written, wait to allow it to fill 
    if func(latest_file) == func(latest_file_pre)+1:
        time.sleep(1)
        
    # Given the new file, update the previous file check for next time
    latest_file_pre = latest_file
    
    #### FOR PLOT DEVELOPMENT ONLY - REMOVE AFTER #####
    if live == False:
        latest_file = pjoin(date_piece,datarun,datarun+'_'+str(buffer_num)+'.mat')
        latest_file_pre == pjoin(date_piece,datarun,datarun+'_'+str(buffer_num)+'.mat')
    
        # For the next loop, iterate
        buffer_num += 1
    
    ## STEP 6: CONVERT & PROCESS DATA 
    
    # Load the newest data (NOTE: Change to vect for full array test)
    mat_contents = sio.loadmat(latest_file)
    data = mat_contents[mat_var][0][:]
    
    # Initialize Variables
    data_real = np.zeros((32,32,len(data)-2))  #Data should be a power/divisible by 2^N where N= number of channels active in the recording buffer. Note Divide by 2 because of the double count and -1 because the first sample (w/o double count) is garbo
    cnt_real = np.zeros((32,32,len(data)-2))
    cnt_pre = 0 #Check for cnt changes, double cnt
    chan_index_pre = 1025 #Check for chan changes, double cnt
    N = 0 # Sample times (DOES NOT ALLOW NON-COLLISION FREE SAMPLES)
    
    for i in range(2,len(data)-1): 
        # Convert bit number into binary
        word = (np.binary_repr(data[i],32))
        
        # Break that binary into it's respective pieces and convert to bit number
        cnt = int(word[12:14],2)
        col = int(word[27:32],2)
        row = int(word[22:27],2)
        chan_index = row*32 + col
        
        # Only record the unique non-double count sample
        if(i==2 or (cnt_pre != cnt or chan_index != chan_index_pre)):
            
            # Sample time only changes when cnt changes
            if cnt != cnt_pre:
                N += 1
                
            # On the occurance the first cnt is not 0, make sure sample time is 0
            if i == 2:
                N = 0
                
            # Update variables
            cnt_pre = cnt
            chan_index_pre = chan_index
            
            # Record pertinent data
            data_real[row][col][N] = int(word[14:22],2)
            cnt_real[row][col][N] = cnt
            
    # Truncate data to be the number of samples       
    num_samples = N
    data_real = data_real[:,:,:N]
    cnt_real = cnt_real[:,:,:N]
    
    ## STEP 7: ANALYSIS FOR PLOTTING 

    # AX2,AX3) Determine the Time of Each Sample
    total_time = num_samples*0.05 #Sampling rate 1/0.05 ms 
    times = np.linspace(0,total_time,num_samples)
    times = times+last_time #If same electrode, update plot begin times
    last_time = times[len(times)-1]
    
    ### NEED TO FIX UPDATE NOW THAT WE ARE PLOTTING 2 ELECTRODES ###
    last_time1 = times[len(times)-1]
    last_time2 = times[len(times)-1]
    cnt_time1 += len(times) #If same electrode, how many total samples are plotted
    cnt_time2 += len(times)
    
    # If same electrode reaches too many samples, clear it
    if cnt_time1 >= max_samples:
        ax[0,1].clear()
        ax[0,1].set_xlabel('Time (ms)',{'fontsize':12})
        ax[0,1].set_ylabel('Amplitude (ADC)',{'fontsize':12})
        ax[0,1].set_title('Current Elec Max Samples',{'fontsize':14})
        cnt_time1 = 0
    if cnt_time2 >= max_samples:
        ax[1,1].clear()
        ax[1,1].set_xlabel('Time (ms)',{'fontsize':12})
        ax[1,1].set_ylabel('Amplitude (ADC)',{'fontsize':12})
        ax[1,1].set_title('Current Elec Max Amplitude',{'fontsize':14})
        ax[1,1].text(1, 1.15, textstr2,transform=ax[0,2].transAxes,fontsize=12,va = 'center',ha='right')
        cnt_time2 = 0
    
    # AX1) Initialize Array Indices
    base = np.arange(1,33)
    rows = [ele-1 for ele in base for i in range(32)]
    temp = [1 for i in range(32)]
    cols = [ele-1 * tele for tele in temp for ele in base]
    
    # AX4,AX5,AX6) Histograms by Buffer
    
    # Counting True-Valued Samples
    sam_cnt = np.count_nonzero(data_real,axis=2)
    incom_sam = np.copy(sam_cnt)
    
    # Average Amplitudes
    mask = np.copy(data_real)
    mask[mask==0] = np.nan
    avg_amp = np.nanmean(mask,axis=2)
    avg_amp = np.nan_to_num(avg_amp,nan=0)
    incom_amp = avg_amp
    
    # Average Unique Channels
    unq_setup = data_real.reshape(1024,len(data_real[0][0]))
    unq_cnt = np.count_nonzero(unq_setup,axis=0)
    
    # Set Up
    pre_amp = amp_per
    pre_cnt = amp_cnt
    amp_div = pre_cnt + incom_sam
    amp_div[amp_div==0] = np.nan
    
    pre_avg_unq = avg_unq
    
    pre_sam = sam_per
    pre_buf_cnt = buf_cnt
    incom_buf_cnt = 1
    sam_div = pre_buf_cnt + incom_buf_cnt
    
    # Calc
    amp_per = np.nan_to_num((pre_cnt*pre_amp+incom_sam*incom_amp)/(amp_div),nan=0)
    amp_cnt = np.nan_to_num(amp_div,nan=0)
    
    unq_per = np.append(unq_per,np.mean(unq_cnt))
    avg_unq = np.nan_to_num((pre_buf_cnt*pre_avg_unq+incom_buf_cnt*len(unq_cnt))/(sam_div),nan=0)
    
    sam_per = np.nan_to_num((pre_buf_cnt*pre_sam+incom_buf_cnt*incom_sam)/(sam_div),nan=0)
    buf_cnt += 1
    
    # Find Channel of Max Amplitude and Max Samples
    fig_elec[0,0] = np.argmax(incom_sam)
    fig_elec[1,0] = np.argmax(incom_amp)
    fig_elec[:,1] = (fig_elec[:,0]/32).astype(int)
    fig_elec[:,2] = (fig_elec[:,0] - (fig_elec[:,1]*32)).astype(int)
    fig_elec = fig_elec.astype(int)
    
    # Reset Any Necessary Plots
    if fig_elec_pre[0]  != fig_elec[0,0]:
        ax[0,1].clear()
        ax[0,1].set_xlabel('Time (ms)',{'fontsize':12})
        ax[0,1].set_ylabel('Amplitude (ADC)',{'fontsize':12})
        ax[0,1].set_title('Current Elec Max Samples',{'fontsize':14})
    if fig_elec_pre[1] != fig_elec[1,0]:
        ax[1,1].clear()
        ax[1,1].set_xlabel('Time (ms)',{'fontsize':12})
        ax[1,1].set_ylabel('Amplitude (ADC)',{'fontsize':12})
        ax[1,1].set_title('Current Elec Max Amplitude',{'fontsize':14})
        ax[1,1].text(1, 1.15, textstr2,transform=ax[0,2].transAxes,fontsize=12,va = 'center',ha='right')
    if buffer_num-1 > 0:
        ax[1,0].clear()
        twin1.clear()
        ax[0,2].clear()
        ax[1,2].clear()
        ax[1,0].set_title('Unique Channels per Sample',{'fontsize':14})
        ax[0,2].set_title('Sample Rate',{'fontsize':14})
        ax[1,2].set_title('Amplitude',{'fontsize':14})
        ax[1,0].set_xlabel('Unique Channel Count',{'fontsize':12})
        ax[1,0].set_ylabel('Number of Samples',{'fontsize':12})
        twin1.set_ylabel('Number of Buffers',{'fontsize':12})
        ax[0,2].set_xlabel('Electrode Number',{'fontsize':12})
        ax[0,2].set_ylabel('Avg. Sample Count per Buffer',{'fontsize':12})
        ax[1,2].set_xlabel('Electrode Number',{'fontsize':12})
        ax[1,2].set_ylabel('Avg. Amplitude per Sample',{'fontsize':12})
    
    # PLOT THE ONES FOR A SINGLE BUFFER
    cm0 = plt.cm.get_cmap('inferno')
    elec_all = np.arange(1024)
    C0 = [cm0(((x-sam_per.min())/(sam_per.max()-sam_per.min()))) for x in sam_per.flatten()]
    ax[0,2].bar(elec_all,sam_per.flatten(),color=C0,edgecolor='black',linewidth=0,align='edge',width=1)
    asp = np.diff(ax[0,2].get_xlim())[0]/np.diff(ax[0,2].get_ylim())[0]
    ax[0,2].set_aspect(asp/squish)
    ax[0,2].set_xlim(left=-0.5,right=1023.5)

    C1 = [cm0(((x-amp_per.min())/(amp_per.max()-amp_per.min()))) for x in amp_per.flatten()]
    ax[1,2].bar(elec_all,amp_per.flatten(),color=C1,edgecolor='black',linewidth=0,align='edge',width=1)
    asp = np.diff(ax[1,2].get_xlim())[0]/np.diff(ax[1,2].get_ylim())[0]
    ax[1,2].set_aspect(asp/squish)
    ax[1,2].set_xlim(left=-0.5,right=1023.5)
    
    hist_unq_cnt, hist_unq = np.histogram(unq_cnt,bins=30)
    lb1 = ax[1,0].bar(hist_unq[:-1],hist_unq_cnt,edgecolor='black',linewidth=1,align='edge',width=(hist_unq[1]-hist_unq[0])/2,label='Current Buffer',color='blue')
    asp = np.diff(ax[1,0].get_xlim())[0]/np.diff(ax[1,0].get_ylim())[0]
    ax[1,0].set_aspect(asp)
    avg_unq_per, hist_avg_unq = np.histogram(unq_per,bins=hist_unq)
    lb2 = twin1.bar(hist_avg_unq[:-1]+((hist_unq[1]-hist_unq[0])/2),avg_unq_per,edgecolor='black',linewidth=1,align='edge',width=(hist_unq[1]-hist_unq[0])/2,label='Prev. Buffer Avg.',color='red')
    ax[1,0].legend(handles=[lb1,lb2],loc='upper left',fontsize=11,edgecolor='black',facecolor='white')
    twin1.spines['left'].set_linewidth(2)
    twin1.spines['left'].set_color('blue')
    twin1.spines['right'].set_linewidth(2)
    twin1.spines['right'].set_color('red')
    twin1.grid(False)
    
    # AX1) Initialize Variables for the Plotting Loop
    size = np.zeros((32,32,0)) # For each dot
    num_sam = np.zeros((32,32,1))
    colors = np.zeros((32,32,0)) # For each dot
    avg_val = np.zeros((32,32,1))
    max_num_temp = np.zeros((2,1))
    min_num_temp = np.zeros((2,1))
    
    # Initialize every time
    x1 = rows
    y1 = cols
    x2 = np.empty(0)
    y2 = np.empty(0)
    x3 = np.empty(0)
    y3 = np.empty(0)
    n = 0
    
    # Start with the Last Bit of Information
    if total_time > real_time_win:
        i = num_samples - int(real_time_win/0.05)
        x2 = np.append(x2,times[0:num_samples-int(real_time_win/0.05)-1])
        y2 = np.append(y2,data_real[fig_elec[0,1],fig_elec[0,2],0:num_samples-int(real_time_win/0.05)-1])
        x3 = np.append(x3, times[0:num_samples-int(real_time_win/0.05)-1])
        y3 = np.append(y3,data_real[fig_elec[1,1],fig_elec[1,2],0:num_samples-int(real_time_win/0.05)-1])
        ax[0,1].plot(x2,y2,color='r', linewidth=0.5)
        ax[1,1].plot(x2,y2,color='r',linewidth=0.5)
        fig.canvas.draw()
    elif total_time <= real_time_win:
        i = 0
        
    text_sam = 'Elec # ' + str(fig_elec[0,0])
    text_amp = 'Elec # ' + str(fig_elec[1,0])
    ax[0,1].annotate(text_sam,(0.01,0.9),xycoords='axes fraction',va='center',fontsize=11,color='blue')
    ax[1,1].annotate(text_amp,(0.01,0.9),xycoords='axes fraction',va='center',fontsize=11,color='blue')
    
    # Calculate plotting values over entire given sample space
    while i < num_samples:
    
        # Determine the non-zero collection of samples in time window and max number of samples in time window
        num_sam[:,:,0] = np.count_nonzero(data_real[:,:,i:i+sample_win],axis=2)
        max_sam = np.max(num_sam[:,:,0])
        min_sam = np.min(num_sam[num_sam!=0])
    
        # Determine average value of samples in time window and max value of samples in time window
        num_sam[num_sam==0] = np.nan
        avg_val[:,:,0] = np.sum(data_real[:,:,i:i+sample_win],axis=2)/num_sam[:,:,0]
        avg_val = np.nan_to_num(avg_val,nan=0)
        
        # Record the average values to represent colors for each time window
        # NOTE: Using append b/c it does not require exact calculations. May be a faster version of append (i.e. concatenate)
        colors = np.append(colors,avg_val,axis=2)
    
        num_sam = np.nan_to_num(num_sam,nan=0)
        # Scale size according to the max value unless that value exceeds the saturation point. Squaring in the scale to emphasize differences
        scale = (max_dot - 15) / (max_sam - min_sam)
        b_add = max_dot - (max_sam*scale)
        size = np.append(size,np.round(num_sam*scale+b_add),axis=2)
        size[size<15] = 10
#         if max_num_temp[0,0] < max_cnt:
#             scale = (max_dot - 10) / (max_num_temp[0,0]**2)
#             size = np.append(size, num_sam*scale + 10, axis = 2)
#         elif max_num_temp[0,0] >= max_cnt:
#             scale = (max_dot - 10) / (max_cnt**2)
#             size = np.append(size, num_sam*scale + 10, axis = 2)
        
        # Clear Scatter Plot
        ax[0,0].clear()
        ax[0,0].scatter(y1,x1,size[:,:,n],c=colors[:,:,n],cmap='plasma')
        ax[0,0].set_title('Current Activity',{'fontsize':14})
        ax[0,0].set_xlim(left=-2,right=34)
        ax[0,0].set_ylim(bottom=-2,top=34)
        ax[0,0].set_aspect('equal')
        axin0 = inset_axes(ax[0,0],height='90%',width='5%',loc='center right',borderpad=0)
        cbar0 = ax[0,0].figure.colorbar(ax[0,0].scatter(y1,x1,size[:,:,n],c=colors[:,:,n],cmap='plasma'),ax=ax[0,0],cax=axin0,pad=0.2,orientation='vertical')
        cbar0.ax.tick_params(labelsize=9)
        cbar0.ax.set_xlabel('Amplitude (ADC)',va='top',ha = 'left',fontsize=11,rotation=0)
    
        # For the electrode plot, append the new data to be plotted
        x2 = np.append(x2,times[i:i+sample_win])
        y2 = np.append(y2, data_real[fig_elec[0,1], fig_elec[0,2], i:i+sample_win])
        x3 = np.append(x3, times[i:i+sample_win])
        y3 = np.append(y3,data_real[fig_elec[1,1],fig_elec[1,2],i:i+sample_win])
    
        # For the electrode plot, set a new xlimit
        ax[0,1].plot(x2,y2,color='r', linewidth=0.5)
        ax[0,1].set_xlim(left=max(0,int(times[i])-5),right=(int(times[i])+time_win+1))
        asp = np.diff(ax[0,1].get_xlim())[0]/np.diff(ax[0,1].get_ylim())[0]
        ax[0,1].set_aspect(asp/1.5)
        ax[1,1].plot(x3,y3,color='r', linewidth=0.5)
        ax[1,1].set_xlim(left=max(0,int(times[i])-5),right=(int(times[i])+time_win+1))
        asp = np.diff(ax[1,1].get_xlim())[0]/np.diff(ax[1,1].get_ylim())[0]
        ax[1,1].set_aspect(asp/1.5)
    
        # This step is fast but most time consuming - may replace with fig.canvas.update()
        fig.canvas.draw()
    
        # Flush events (equivalent to time.sleep or plt.pause but faster)
        fig.canvas.flush_events()
        
        cbar0.remove()
            
        data_dir = glob.glob(pjoin(date_piece,datarun)+file_type)

        # In case multiple files coming in at once, hold 100ms (not likely)
        #time.sleep(0.1)

        # Choose the file by the largest file number
        latest_file = max(data_dir, key=func)

        # If the latest file is the same as the previously plotted file, hold
        if latest_file == latest_file_pre:
            pass
        else:
            break
    
        # Use time.sleep if you want to plot really slowly you weirdo
        #time.sleep(1)
    
        # Update the variables
        i += sample_win
        n += 1
        
        # Once at the end, go back to the beginning
        if i >= num_samples:
            break
        
    

        
        
    ## STEP 7: PLOT THAT DATA
    # Note: This can be applied at the end of the last cell, but Jupyter
    # Notebooks gets angry about this even though the plotting goes on as normal

    # NOTE: This needs updated to hold on the last figure of true data while we are waiting for 
    # another file
    

    # Plot for the entire length of incoming data
    
        
# NOTE: Put in interrupt on updating plot to look for new data coming in
# Put in a Pause Button

2021-08-23-0/data010/data010_31.mat


<ipython-input-10-43eb4a4e6710>:123: RuntimeWarning: Mean of empty slice
  avg_amp = np.nanmean(mask,axis=2)


2021-08-23-0/data010/data010_31.mat


<ipython-input-10-43eb4a4e6710>:123: RuntimeWarning: Mean of empty slice
  avg_amp = np.nanmean(mask,axis=2)


2021-08-23-0/data010/data010_31.mat


...

2021-08-23-0/data010/data010_31.mat


<ipython-input-10-43eb4a4e6710>:123: RuntimeWarning: Mean of empty slice
  avg_amp = np.nanmean(mask,axis=2)


2021-08-23-0/data010/data010_31.mat


<ipython-input-10-43eb4a4e6710>:123: RuntimeWarning: Mean of empty slice
  avg_amp = np.nanmean(mask,axis=2)


KeyboardInterrupt: 

In [ ]:
sio.loadmat(latest_file)

In [28]:
print(np.max(sam_per))

1182.8181818181818


In [61]:
print(hist_unq)
print(hist_unq_cnt)

[ 0.          0.08268734  0.16537468  0.24806202  0.33074935  0.41343669
  0.49612403  0.57881137  0.66149871  0.74418605  0.82687339  0.90956072
  0.99224806  1.0749354   1.15762274  1.24031008  1.32299742  1.40568475
  1.48837209  1.57105943  1.65374677  1.73643411  1.81912145  1.90180879
  1.98449612  2.06718346  2.1498708   2.23255814  2.31524548  2.39793282
  2.48062016  2.56330749  2.64599483  2.72868217  2.81136951  2.89405685
  2.97674419  3.05943152  3.14211886  3.2248062   3.30749354  3.39018088
  3.47286822  3.55555556  3.63824289  3.72093023  3.80361757  3.88630491
  3.96899225  4.05167959  4.13436693  4.21705426  4.2997416   4.38242894
  4.46511628  4.54780362  4.63049096  4.71317829  4.79586563  4.87855297
  4.96124031  5.04392765  5.12661499  5.20930233  5.29198966  5.374677
  5.45736434  5.54005168  5.62273902  5.70542636  5.7881137   5.87080103
  5.95348837  6.03617571  6.11886305  6.20155039  6.28423773  6.36692506
  6.4496124   6.53229974  6.61498708  6.69767442  6.7

In [60]:
hist_avg_unq[:-1]+((hist_unq[1]-hist_unq[0])/2)

array([ 0.04134367,  0.12403101,  0.20671835,  0.28940568,  0.37209302,
        0.45478036,  0.5374677 ,  0.62015504,  0.70284238,  0.78552972,
        0.86821705,  0.95090439,  1.03359173,  1.11627907,  1.19896641,
        1.28165375,  1.36434109,  1.44702842,  1.52971576,  1.6124031 ,
        1.69509044,  1.77777778,  1.86046512,  1.94315245,  2.02583979,
        2.10852713,  2.19121447,  2.27390181,  2.35658915,  2.43927649,
        2.52196382,  2.60465116,  2.6873385 ,  2.77002584,  2.85271318,
        2.93540052,  3.01808786,  3.10077519,  3.18346253,  3.26614987,
        3.34883721,  3.43152455,  3.51421189,  3.59689922,  3.67958656,
        3.7622739 ,  3.84496124,  3.92764858,  4.01033592,  4.09302326,
        4.17571059,  4.25839793,  4.34108527,  4.42377261,  4.50645995,
        4.58914729,  4.67183463,  4.75452196,  4.8372093 ,  4.91989664,
        5.00258398,  5.08527132,  5.16795866,  5.25064599,  5.33333333,
        5.41602067,  5.49870801,  5.58139535,  5.66408269,  5.74

In [49]:
print(unq_cnt)

[25  0  2 27  0  1 27  0  4 29  0  1 30 30  0  3 29  0  1 28  0  2 29  0
  1 31 28  0  1 29  0  1 30 30 30 31 31  0  1 30 29  0  1 27  0  3 30 30
 31 31 27  0  4 29  0  1 27  0  1 28 28 30  1  1 29  0  1 28  0  1  1  1
 28  0  1 25  1  1  0  1  0  1 26  0  2 24  0  2  0  3 26  1  2  0  1 25
  1  2  0  1 26  0  1  0  1 23  0  2  0  5 27 25  0  2 26  0  1 26  0  1
  0  1 27 27 30 27 27  1  3 29  0  1 30 31  0  1 30 31 31 31 30  1  1 31
 31  0  1 31 30 29  0  1 29  0  1 31  0  1 30 29  0  1 30 31 30  0  1  1
  1 27  0  3  1  1 27  0  3 28  1  1 29  0  3 29 27  0  2 29  0  1 26  0
  4 28  0  2 28  1  2 29  0  2 29  0  3 27  0  3 29 25  0  7 29  0  3 28
  0  2 30  0  2 26  0  4 27  0  4 27  0  3 30  0  2 26  0  5 29  0  1 29
  0  3 29  0  4 26  0  6  1  1 26  0  2 25  0  5 29  0  1 28  0  3 28  0
  4  1  1 26  0  3 29  0  2  1  2 26  0  4  1  2 26  0  3 29  0  2 27  0
  4 26  0  4  1  1 28  0  1 28  0  1 30 28  0  4 28  0  3 27  0  4 30  0
  1 29 30 31 31 32 32 32 31 30  0  3 30  0  2 30 31